In [1]:
!pip install flask flask-ngrok

In [2]:
!pip install pyngrok
!ngrok authtoken 2uaLv7XxRvNzWda8KuxYB3eT6Po_6Puu2sGxtqxf97ETaNCg8

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [11]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok
from pyngrok import ngrok
import tensorflow as tf
import numpy as np
import cv2
import base64
import io
from PIL import Image

app = Flask(__name__)
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

model = tf.keras.models.load_model("/content/Rudra3_xray_Model.h5")
classes = ["Covid-19", "Lung Opacity", "Normal","Pneumonia", "Tuberculosis"]

@app.route('/predict', methods=['POST'])
def predict():
    try:
        data = request.json['image']
        image_bytes = base64.b64decode(data)
        image = Image.open(io.BytesIO(image_bytes))
        if image.mode != 'L':
            image = image.convert('L')

        image = image.resize((224, 224))
        image = np.array(image)
        image=image/255
        img_array = tf.keras.utils.img_to_array(image)
        img_array = np.expand_dims(img_array, axis=0)
        prediction = model.predict(img_array)
        prediction = np.argmax(prediction)
        predicted_class = classes[prediction]
        print(predicted_class)


        return jsonify({"disease": predicted_class})

    except Exception as e:
        return jsonify({"error": str(e)})

if __name__ == '__main__':
    app.run()


Public URL: https://4721-35-237-185-90.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 11:14:54] "POST /predict HTTP/1.1" 200 -


Lung Opacity
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


...

Normal
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 163ms/step


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 11:15:19] "POST /predict HTTP/1.1" 200 -


Pneumonia
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


INFO:werkzeug:127.0.0.1 - - [27/Mar/2025 11:15:27] "POST /predict HTTP/1.1" 200 -


Tuberculosis
